# Retrieve data from Wiki page

In [3]:
# import html and requests to scrape the data from the webpage in html format
!pip install lxml
from lxml import html
import requests

# use the wikipedia url to save the html tree into a python variable

wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
wiki_page = requests.get(wiki_url)
tree = html.fromstring(wiki_page.content)

# parse through table rows in html tree (xpath contains table row)
tr_elements = tree.xpath('//tr')

#empty list for table
tab=[]
i=0

#For each row in the table, store each header for the column name and an empty list for each column
for t in tr_elements[0]:
    i+=1
    name=t.text_content()
    tab.append((name,[]))
    
print(tab)

     |████████████████████████████████| 5.8MB 29.1MB/s eta 0:00:01
[('Postcode', []), ('Borough', []), ('Neighbourhood\n', [])]


In [4]:
#Since our first row is the header, data is stored on the second row onwards
for j in range(1,len(tr_elements)):
    #T is our j'th row
    T=tr_elements[j]
    
    #If row is not of size 3, the //tr data is not from our table 
    if len(T)!=3:
        break
    
    #i is the index of our column
    i=0
    
    #Iterate through each element of the row
    for t in T.iterchildren():
        data=t.text_content() 
        #Check if row is empty
        if i>0:
        #Convert any numerical value to integers
            try:
                data=int(data)
            except:
                pass
        #Append the data to the empty list of the i'th column
        tab[i][1].append(data)
        #Increment i for the next column
        i+=1

# Store data in data frames

In [4]:
# convert list to dictionary

dict = {title:column for (title,column) in tab}

# store dictionary in pandas dataframe

import pandas as pd

tor_df = pd.DataFrame(dict)
print(tor_df.shape)
print(tor_df.head())

(288, 3)
  Postcode           Borough     Neighbourhood\n
0      M1A      Not assigned      Not assigned\n
1      M2A      Not assigned      Not assigned\n
2      M3A        North York         Parkwoods\n
3      M4A        North York  Victoria Village\n
4      M5A  Downtown Toronto      Harbourfront\n


# Clean data

In [ ]:
#The data in the dataframe needs to be organized. The first step is to rename the "PostCode" and "Neighbourhood\n" columns to "Postal Code" and "Neighborhood"

# rename Postal Code and Neighborhood column
tor_df.rename(columns={'Postcode':'PostalCode', 'Neighbourhood\n':'Neighborhood'}, inplace=True)
tor_df.head()

In [ ]:
# All of the values in the third column end with a "\n" due to the html formatting specifying a new line or row in the table. This needs to be deleted in the dataframe.

# replace the "\n" in the third column with ""
tor_df['Neighborhood'] = tor_df['Neighborhood'].str.replace('\n','')
tor_df.head()

In [ ]:
# delete all of the values in the Borough column that equal "Not assigned".

# count unnasigned values in borough
print(tor_df[tor_df.Borough == 'Not assigned'].count())
print('--------------')

# ignore unassigned values in borough
tor_df = tor_df[tor_df.Borough != 'Not assigned']

# double check there are no not assigned values remaining in borough
print(tor_df[tor_df.Borough == 'Not assigned'].count())

group the data by postal code and join the neighborhoods with the same postal code separated by a comma. I used an aggregate function for this but I could not figure out how to keep the borough column in the dataframe without concatenating the same boroughs together as well.


In [ ]:
# group by postal code and join the neighborhoods with the same postal code separated by a ','
df_temp = tor_df.groupby('PostalCode').agg({'Neighborhood':', '.join})
df_temp.reset_index(inplace=True)
df_temp.head(10)

To introduce the borough column back into the dataframe, I merge the temporary grouped dataframe from the previous cell with the original dataframe then delete the duplicate values. I then reorder the columns and reset the index.

In [ ]:
# merge the dataframes back together to get boroughs
df_tor = pd.merge(df_temp[['PostalCode','Neighborhood']], tor_df[['PostalCode', 'Borough']], left_on = 'PostalCode', right_on = 'PostalCode', how = 'left')

#drop duplicate postal code values
df_tor.drop_duplicates(subset='PostalCode', inplace=True)

#reorder colums and reset index
df_tor = df_tor[['PostalCode', 'Borough', 'Neighborhood']]
df_tor.reset_index(drop = True, inplace = True)

df_tor.head(10)

check if there are any unassigned values in neighborhood column that weren't dropped when I took out the unassigned values from the borough column. Turns out there is one unassigned value so I replace it with the corresponding borough value.

In [ ]:
# find if there are any not assigned values for neighborhood
df_tor[df_tor.Neighborhood == 'Not assigned']

In [ ]:
# replace not assigned value for neighborhood with the borough
df_tor.loc[df_tor.Neighborhood == 'Not assigned', 'Neighborhood'] = df_tor.loc[df_tor.Neighborhood == 'Not assigned', 'Borough']
df_tor[df_tor.PostalCode == 'M7A']

### print the shape of the dataframe.

In [ ]:
print(df_tor.shape)

In [ ]:
#print the entire table to prove that my data contains all of the boroughs in Toronto and not just Scarborough
df_tor